In [7]:
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

The Narrative Analysis tool should now be fully functional, allowing you to:

Upload JSON files containing narrative text
Analyze the narrative structure (timeline, plotline, and storyline)
Visualize the results with interactive charts
Generate and download a comprehensive PDF report

The GUI will:
Validate JSON format and provide specific error messages
Check if the JSON contains actual narrative text
Provide more detailed feedback at each step of the analysis
Include a help button that explains the expected file format with examples
The tool should be user-friendly and help users understand what went wrong if they upload an incorrect file format.

If you want to test it with a sample narrative, you can create a simple JSON file with this structure:

{
  "title": "The Three Little Pigs",
  "text": "Once upon a time, there were three little pigs. The first pig built a house of straw. The second pig built a house of sticks. The third pig built a house of bricks. One day, a big bad wolf came to the first pig's house. He huffed and puffed and blew the house down. The first pig ran to the second pig's house. The wolf followed and blew down the second house too. Both pigs ran to the third pig's house. The wolf tried to blow down the brick house, but he couldn't. He tried to enter through the chimney, but the third pig had a pot of boiling water. The wolf fell into the water and ran away. The three pigs lived happily ever after."
}

In [ ]:
# Install required packages
import sys
import subprocess
import pkg_resources

required_packages = ['pandas', 'numpy', 'matplotlib', 'seaborn', 'networkx', 
                    'ipywidgets', 'nltk', 'spacy', 'fpdf', 'pillow']

installed = {pkg.key for pkg in pkg_resources.working_set}
missing = [pkg for pkg in required_packages if pkg.lower() not in installed]

if missing:
    print(f"Installing missing packages: {missing}")
    subprocess.check_call([sys.executable, '-m', 'pip', 'install'] + missing)
    
    # Install spacy model separately
    if 'spacy' in missing:
        subprocess.check_call([sys.executable, '-m', 'spacy', 'download', 'en_core_web_sm'])


In [7]:
!pip install --upgrade numpy==1.26.4

   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ----------------------- ---------------- 9.2/15.8 MB 51.8 MB/s eta 0:00:01
   ---------------------------------------  15.7/15.8 MB 45.0 MB/s eta 0:00:01
   ---------------------------------------- 15.8/15.8 MB 36.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4



  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


First, let's run the imports and setup:

In [1]:

# Now import all required packages
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output, FileLink
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy
import re
from datetime import datetime
import io
from fpdf import FPDF
import base64
from PIL import Image
import tempfile
import os

# Download necessary NLTK resources
try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('corpora/stopwords')
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('punkt')
    nltk.download('stopwords')
    nltk.download('wordnet')

# Load spaCy model
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    print("Downloading spaCy model...")
    import sys
    !{sys.executable} -m spacy download en_core_web_sm
    nlp = spacy.load("en_core_web_sm")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lissa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lissa\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lissa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Now, let's define the NarrativeAnalyzer class:

In [2]:
class NarrativeAnalyzer:
    def __init__(self):
        self.text = ""
        self.sentences = []
        self.processed_text = ""
        self.events = []
        self.timeline = []
        self.plotline = {}
        self.storyline = {}
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))
        
    def load_data(self, file_content):
        """Load text data from JSON content"""
        try:
            data = json.loads(file_content)
            
            # Handle different JSON structures
            if isinstance(data, dict):
                if 'text' in data:
                    self.text = data['text']
                elif 'content' in data:
                    self.text = data['content']
                else:
                    # Use the first string value found
                    text_found = False
                    for key, value in data.items():
                        if isinstance(value, str) and len(value) > 100:  # Assuming text is reasonably long
                            self.text = value
                            text_found = True
                            break
                    
                    if not text_found:
                        raise ValueError("No text field found in JSON. Expected 'text' or 'content' field.")
            elif isinstance(data, list):
                # Concatenate all string items or text fields
                text_parts = []
                for item in data:
                    if isinstance(item, str):
                        text_parts.append(item)
                    elif isinstance(item, dict) and ('text' in item or 'content' in item):
                        text_parts.append(item.get('text', item.get('content', '')))
                self.text = ' '.join(text_parts)
                
                if not text_parts:
                    raise ValueError("No text content found in JSON array. Expected strings or objects with 'text' field.")
            else:
                raise ValueError(f"Unexpected JSON structure. Expected object or array, got {type(data).__name__}.")
            
            if not self.text:
                raise ValueError("No text content found in the JSON file")
                
            return True
        except json.JSONDecodeError as e:
            print(f"JSON parsing error: {e}")
            return False
        except ValueError as e:
            print(f"Data error: {e}")
            return False
        except Exception as e:
            print(f"Unexpected error: {e}")
            return False
       
    
    def preprocess_text(self):
        """Preprocess the text for NLP analysis"""
        if not self.text:
            return False
        
        # Tokenize into sentences
        self.sentences = sent_tokenize(self.text)
        
        # Process each sentence
        processed_sentences = []
        for sentence in self.sentences:
            # Tokenize words
            words = word_tokenize(sentence)
            
            # Remove stopwords and lemmatize
            filtered_words = [self.lemmatizer.lemmatize(word.lower()) 
                             for word in words 
                             if word.lower() not in self.stop_words and word.isalnum()]
            
            processed_sentences.append(' '.join(filtered_words))
        
        self.processed_text = ' '.join(processed_sentences)
        return True
    
    def extract_events(self):
        """Extract events from the text using spaCy"""
        if not self.sentences:
            return False
        
        self.events = []
        for i, sentence in enumerate(self.sentences):
            doc = nlp(sentence)
            
            # Extract events (verbs and their arguments)
            for token in doc:
                if token.pos_ == "VERB":
                    # Get subject
                    subjects = [subj.text for subj in token.head.children if subj.dep_ in ("nsubj", "nsubjpass")]
                    subject = subjects[0] if subjects else ""
                    
                    # Get object
                    objects = [obj.text for obj in token.children if obj.dep_ in ("dobj", "pobj")]
                    obj = objects[0] if objects else ""
                    
                    # Get time expressions
                    time_entities = [ent.text for ent in doc.ents if ent.label_ in ("DATE", "TIME")]
                    time = time_entities[0] if time_entities else ""
                    
                    # Create event
                    event = {
                        "sentence_id": i,
                        "sentence": sentence,
                        "verb": token.text,
                        "subject": subject,
                        "object": obj,
                        "time": time
                    }
                    self.events.append(event)
        
        return len(self.events) > 0


Let's continue with the analysis methods:

In [3]:
    def analyze_timeline(self):
        """Analyze the timeline of events"""
        if not self.events:
            return False
        
        # Sort events by sentence_id to maintain chronological order
        sorted_events = sorted(self.events, key=lambda x: x["sentence_id"])
        
        # Create timeline
        self.timeline = []
        for event in sorted_events:
            timeline_event = {
                "event": f"{event['subject']} {event['verb']} {event['object']}".strip(),
                "time": event["time"] if event["time"] else "Unspecified",
                "sentence": event["sentence"]
            }
            self.timeline.append(timeline_event)
        
        return len(self.timeline) > 0
    
    def analyze_plotline(self):
        """Analyze the plotline using Vossen's framework"""
        if not self.events:
            return False
        
        # Initialize plotline components
        self.plotline = {
            "exposition": [],
            "rising_action": [],
            "climax": [],
            "falling_action": [],
            "resolution": []
        }
        
        # Simple heuristic: divide events into 5 parts
        total_events = len(self.events)
        section_size = max(1, total_events // 5)
        
        # Assign events to plotline components
        for i, event in enumerate(self.events):
            event_summary = f"{event['subject']} {event['verb']} {event['object']}".strip()
            
            if i < section_size:
                self.plotline["exposition"].append(event_summary)
            elif i < section_size * 2:
                self.plotline["rising_action"].append(event_summary)
            elif i < section_size * 3:
                self.plotline["climax"].append(event_summary)
            elif i < section_size * 4:
                self.plotline["falling_action"].append(event_summary)
            else:
                self.plotline["resolution"].append(event_summary)
        
        return True
    
    def analyze_storyline(self):
        """Analyze the storyline using narratology frameworks"""
        if not self.events:
            return False
        
        # Initialize storyline components based on Caselli and Segers' framework
        self.storyline = {
            "characters": {},
            "settings": [],
            "conflicts": [],
            "themes": [],
            "narrative_arcs": []
        }
        
        # Extract characters (subjects and objects)
        characters = {}
        for event in self.events:
            if event["subject"] and len(event["subject"]) > 1:
                if event["subject"] not in characters:
                    characters[event["subject"]] = {"actions": [], "mentions": 0}
                characters[event["subject"]]["mentions"] += 1
                characters[event["subject"]]["actions"].append(event["verb"])
            
            if event["object"] and len(event["object"]) > 1:
                if event["object"] not in characters:
                    characters[event["object"]] = {"actions": [], "mentions": 0}
                characters[event["object"]]["mentions"] += 1
        
        # Keep only significant characters (mentioned more than once)
        self.storyline["characters"] = {k: v for k, v in characters.items() if v["mentions"] > 1}
        
        # Extract settings (time expressions)
        settings = set()
        for event in self.events:
            if event["time"]:
                settings.add(event["time"])
        self.storyline["settings"] = list(settings)
        
        # Simple conflict detection (negative verbs or emotional content)
        conflict_verbs = ["fight", "argue", "disagree", "oppose", "conflict", "battle", "struggle"]
        for event in self.events:
            for conflict_verb in conflict_verbs:
                if conflict_verb in event["verb"].lower():
                    self.storyline["conflicts"].append(
                        f"{event['subject']} {event['verb']} {event['object']}".strip()
                    )
        
        # Identify themes (most common verbs)
        all_verbs = [event["verb"].lower() for event in self.events]
        verb_freq = {}
        for verb in all_verbs:
            if verb not in verb_freq:
                verb_freq[verb] = 0
            verb_freq[verb] += 1
        
        # Top 5 verbs as themes
        top_verbs = sorted(verb_freq.items(), key=lambda x: x[1], reverse=True)[:5]
        self.storyline["themes"] = [verb for verb, freq in top_verbs]
        
        # Narrative arcs (simplified)
        self.storyline["narrative_arcs"] = [
            "Introduction of characters",
            "Setting establishment",
            "Conflict development",
            "Rising tension",
            "Resolution"
        ]
        
        return True


Now, let's add the visualization methods:

In [4]:
    def visualize_timeline(self):
        """Create a visualization of the timeline"""
        if not self.timeline:
            return None
        
        # Create a DataFrame for the timeline
        df = pd.DataFrame(self.timeline)
        
        # Create a figure
        plt.figure(figsize=(12, 6))
        
        # Plot events on a timeline
        for i, event in enumerate(self.timeline):
            plt.scatter(i, 1, s=100, color='blue')
            plt.text(i, 1.1, event["event"], rotation=45, ha='right', fontsize=8)
        
        plt.yticks([])
        plt.xlabel('Event Sequence')
        plt.title('Narrative Timeline')
        plt.tight_layout()
        
        # Save the figure to a bytes buffer
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        plt.close()
        
        return buf
    
    def visualize_plotline(self):
        """Create a visualization of the plotline"""
        if not self.plotline:
            return None
        
        # Count events in each plot component
        plot_counts = {k: len(v) for k, v in self.plotline.items()}
        
        # Create a figure
        plt.figure(figsize=(12, 6))
        
        # Plot the arc
        x = np.arange(len(plot_counts))
        y = [plot_counts["exposition"], 
             plot_counts["rising_action"], 
             plot_counts["climax"],
             plot_counts["falling_action"], 
             plot_counts["resolution"]]
        
        # Create a smooth curve
        x_smooth = np.linspace(0, len(x)-1, 100)
        y_smooth = np.interp(x_smooth, x, y)
        
        plt.plot(x_smooth, y_smooth, 'b-', linewidth=2)
        plt.fill_between(x_smooth, y_smooth, alpha=0.3)
        
        plt.xticks(x, list(plot_counts.keys()), rotation=45)
        plt.ylabel('Number of Events')
        plt.title('Narrative Plot Structure')
        plt.tight_layout()
        
        # Save the figure to a bytes buffer
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        plt.close()
        
        return buf
    
    def visualize_storyline(self):
        """Create a visualization of the storyline"""
        if not self.storyline or not self.storyline["characters"]:
            return None
        
        # Create a character network
        G = nx.Graph()
        
        # Add character nodes
        for character in self.storyline["characters"]:
            G.add_node(character, size=self.storyline["characters"][character]["mentions"] * 100)
        
        # Add edges between characters that appear in the same events
        character_pairs = []
        for event in self.events:
            if event["subject"] in self.storyline["characters"] and event["object"] in self.storyline["characters"]:
                character_pairs.append((event["subject"], event["object"]))
        
        # Count frequency of character interactions
        edge_weights = {}
        for pair in character_pairs:
            if pair not in edge_weights:
                edge_weights[pair] = 0
            edge_weights[pair] += 1
        
        # Add weighted edges
        for pair, weight in edge_weights.items():
            G.add_edge(pair[0], pair[1], weight=weight)
        
        # Create a figure
        plt.figure(figsize=(12, 8))
        
        # Get node sizes
        node_sizes = [G.nodes[node]['size'] for node in G.nodes]
        
        # Get edge weights
        edge_weights = [G.edges[edge]['weight'] for edge in G.edges]
        
        # Draw the network
        pos = nx.spring_layout(G, seed=42)
        nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color='lightblue')
        nx.draw_networkx_edges(G, pos, width=edge_weights, alpha=0.7)
        nx.draw_networkx_labels(G, pos, font_size=10)
        
        plt.title('Character Relationship Network')
        plt.axis('off')
        plt.tight_layout()
        
        # Save the figure to a bytes buffer
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        plt.close()
        
        return buf
    
    def generate_report(self):
        """Generate a PDF report of the analysis"""
        # Create a PDF
        pdf = FPDF()
        pdf.add_page()
        
        # Set font
        pdf.set_font("Arial", "B", 16)
        pdf.cell(0, 10, "Computational Narrative Analysis Report", ln=True, align="C")
        pdf.ln(10)
        
        # Add timeline analysis
        pdf.set_font("Arial", "B", 14)
        pdf.cell(0, 10, "1. Timeline Event Analysis", ln=True)
        pdf.set_font("Arial", "", 12)
        pdf.multi_cell(0, 10, "The timeline analysis identifies key events in chronological order, showing how the narrative unfolds over time.")
        
        # Add timeline events
        pdf.set_font("Arial", "I", 12)
        for i, event in enumerate(self.timeline[:10]):  # Limit to first 10 events
            pdf.multi_cell(0, 10, f"{i+1}. {event['event']} ({event['time']})")
        
        # Add timeline visualization
        timeline_img = self.visualize_timeline()
        if timeline_img:
            # Save the image temporarily
            with tempfile.NamedTemporaryFile(delete=False, suffix='.png') as tmp:
                tmp_name = tmp.name
                img = Image.open(timeline_img)
                img.save(tmp_name)
            
            # Add image to PDF
            pdf.add_page()
            pdf.image(tmp_name, x=10, y=30, w=180)
            pdf.ln(120)  # Space for the image
            
            # Clean up
            os.unlink(tmp_name)
        
        # Add plotline analysis
        pdf.add_page()
        pdf.set_font("Arial", "B", 14)
        pdf.cell(0, 10, "2. Plotline Analysis", ln=True)
        pdf.set_font("Arial", "", 12)
        pdf.multi_cell(0, 10, "The plotline analysis breaks down the narrative into the traditional five-act structure based on Vossen's framework.")
        
        # Add plotline components
        for component, events in self.plotline.items():
            pdf.set_font("Arial", "B", 12)
            pdf.cell(0, 10, f"{component.replace('_', ' ').title()}:", ln=True)
            pdf.set_font("Arial", "", 12)
            for i, event in enumerate(events[:3]):  # Limit to first 3 events per component
                pdf.multi_cell(0, 10, f"- {event}")
        
        # Add plotline visualization
        plotline_img = self.visualize_plotline()
        if plotline_img:
            # Save the image temporarily
            with tempfile.NamedTemporaryFile(delete=False, suffix='.png') as tmp:
                tmp_name = tmp.name
                img = Image.open(plotline_img)
                img.save(tmp_name)
            
            # Add image to PDF
            pdf.add_page()
            pdf.image(tmp_name, x=10, y=30, w=180)
            pdf.ln(120)  # Space for the image
            
            # Clean up
            os.unlink(tmp_name)
        
        # Add storyline analysis
        pdf.add_page()
        pdf.set_font("Arial", "B", 14)
        pdf.cell(0, 10, "3. Storyline Analysis", ln=True)
        pdf.set_font("Arial", "", 12)
        pdf.multi_cell(0, 10, "The storyline analysis examines characters, settings, conflicts, themes, and narrative arcs based on Caselli and Segers' narratology framework.")
        
        # Add characters
        pdf.set_font("Arial", "B", 12)
        pdf.cell(0, 10, "Characters:", ln=True)
        pdf.set_font("Arial", "", 12)
        for character, info in list(self.storyline["characters"].items())[:5]:  # Limit to first 5 characters
            pdf.multi_cell(0, 10, f"- {character}: mentioned {info['mentions']} times")
        
        # Add settings
        pdf.set_font("Arial", "B", 12)
        pdf.cell(0, 10, "Settings:", ln=True)
        pdf.set_font("Arial", "", 12)
        for setting in self.storyline["settings"][:5]:  # Limit to first 5 settings
            pdf.multi_cell(0, 10, f"- {setting}")
        
        # Add themes
        pdf.set_font("Arial", "B", 12)
        pdf.cell(0, 10, "Themes:", ln=True)
        pdf.set_font("Arial", "", 12)
        for theme in self.storyline["themes"]:
            pdf.multi_cell(0, 10, f"- {theme}")
        
        # Add storyline visualization
        storyline_img = self.visualize_storyline()
        if storyline_img:
            # Save the image temporarily
            with tempfile.NamedTemporaryFile(delete=False, suffix='.png') as tmp:
                tmp_name = tmp.name
                img = Image.open(storyline_img)
                img.save(tmp_name)
            
            # Add image to PDF
            pdf.add_page()
            pdf.image(tmp_name, x=10, y=30, w=180)
            pdf.ln(120)  # Space for the image
            
            # Clean up
            os.unlink(tmp_name)
        
        # Create a temporary file to save the PDF
        with tempfile.NamedTemporaryFile(delete=False, suffix='.pdf') as tmp:
            tmp_name = tmp.name
            pdf.output(tmp_name)
        
        return tmp_name



Now, let's implement the GUI:

In [24]:
# Create the GUI for the application
# Create an updated GUI function with help button and improved error handling
def create_narrative_analysis_gui():
    # Create the analyzer
    analyzer = NarrativeAnalyzer()
    
    # Create widgets
    header = widgets.HTML(
        value="<h1>Computational Narrative Analysis</h1>"
              "<p>Upload a JSON file containing narrative text for analysis.</p>"
    )
    
    file_upload = widgets.FileUpload(
        accept='.json',
        multiple=False,
        description='Upload JSON:',
        layout=widgets.Layout(width='300px')
    )
    
    analyze_button = widgets.Button(
        description='Analyze Text',
        button_style='primary',
        disabled=True,
        layout=widgets.Layout(width='150px')
    )
    
    download_button = widgets.Button(
        description='Download Report',
        button_style='success',
        disabled=True,
        layout=widgets.Layout(width='150px')
    )
    
    # Add help button
    help_button = widgets.Button(
        description='Help',
        button_style='info',
        layout=widgets.Layout(width='100px')
    )
    
    status_output = widgets.Output()
    help_output = widgets.Output()
    result_output = widgets.Output()
    
    # Create tabs for different analyses
    tab_titles = ['Timeline', 'Plotline', 'Storyline']
    children = [widgets.Output() for _ in range(len(tab_titles))]
    tab = widgets.Tab()
    tab.children = children
    for i in range(len(tab_titles)):
        tab.set_title(i, tab_titles[i])
    
    # Define callback functions
    def on_file_upload_change(change):
        if file_upload.value:
            analyze_button.disabled = False
        else:
            analyze_button.disabled = True
    
    def on_analyze_button_click(b):
        with status_output:
            clear_output()
            print("Analyzing text...")
        logging.info("status output cleared")
        
        # Clear previous results
        for child in tab.children:
            with child:
                clear_output()
        logging.info("tab output cleared")
        
        
        with result_output:
            clear_output()
        logging.info("results output cleared")
        
        
        # Get file content
        logging.info(file_upload.value)
        logging.info(file_upload.value[0])
        file_content = str(next(iter(file_upload.value[0].content.tobytes())))
        logging.info(type(file_content))
        logging.info(file_content)
        
        
        # Validate JSON format first
        try:
            json.loads(file_content)
            logging.info("json loaded")
        
        except json.JSONDecodeError as e:
            with status_output:
                clear_output()
                print(f"❌ Error: Invalid JSON format. Please check your file.")
                print(f"Details: {str(e)}")
                print("\nExpected format examples:")
                print('{"text": "Your narrative text here..."}')
                print('{"content": "Your narrative text here..."}')
                print('{"title": "Story Title", "text": "Your narrative text here..."}')
            return
        
        # Load and analyze the data
        with status_output:
            if analyzer.load_data(file_content):
                print("✅ Data loaded successfully.")
                
                # Check if text was actually found in the JSON
                if not analyzer.text or len(analyzer.text) < 50:  # Arbitrary minimum length
                    clear_output()
                    print("❌ Error: No substantial text content found in the JSON file.")
                    print("\nYour JSON file should contain a text field with narrative content.")
                    print("Expected format examples:")
                    print('{"text": "Once upon a time..."}')
                    print('{"content": "Once upon a time..."}')
                    return
                    
                if analyzer.preprocess_text():
                    print("✅ Text preprocessed successfully.")
                    
                    if analyzer.extract_events():
                        print(f"✅ Extracted {len(analyzer.events)} events.")
                        
                        if analyzer.analyze_timeline():
                            print("✅ Timeline analysis complete.")
                            
                            if analyzer.analyze_plotline():
                                print("✅ Plotline analysis complete.")
                                
                                if analyzer.analyze_storyline():
                                    print("✅ Storyline analysis complete.")
                                    download_button.disabled = False
                                else:
                                    print("❌ Error: Storyline analysis failed. The narrative may not have clear character relationships.")
                            else:
                                print("❌ Error: Plotline analysis failed. The narrative may not have enough events to form a plot structure.")
                        else:
                            print("❌ Error: Timeline analysis failed. The narrative may not have a clear sequence of events.")
                    else:
                        print("❌ Error: No events extracted from the text. The narrative may not contain identifiable actions or events.")
                else:
                    print("❌ Error: Text preprocessing failed. The text may be too short or in an unsupported format.")
            else:
                print("❌ Error: Failed to load data from the JSON file.")
                print("\nPlease ensure your JSON file contains narrative text in one of these formats:")
                print('{"text": "Your narrative text here..."}')
                print('{"content": "Your narrative text here..."}')
                print('{"title": "Story Title", "text": "Your narrative text here..."}')
        
        # Display results in tabs
        # Timeline tab
        with tab.children[0]:
            if analyzer.timeline:
                # Display timeline visualization
                timeline_img = analyzer.visualize_timeline()
                if timeline_img:
                    display(HTML("<h3>Timeline Visualization</h3>"))
                    display(Image.open(timeline_img))
                
                # Display timeline events
                display(HTML("<h3>Timeline Events</h3>"))
                timeline_df = pd.DataFrame(analyzer.timeline)
                display(timeline_df[['event', 'time']])
            else:
                display(HTML("<p>No timeline data available.</p>"))
        
        # Plotline tab
        with tab.children[1]:
            if analyzer.plotline:
                # Display plotline visualization
                plotline_img = analyzer.visualize_plotline()
                if plotline_img:
                    display(HTML("<h3>Plotline Visualization</h3>"))
                    display(Image.open(plotline_img))
                
                # Display plotline components
                display(HTML("<h3>Plotline Components</h3>"))
                for component, events in analyzer.plotline.items():
                    display(HTML(f"<h4>{component.replace('_', ' ').title()}</h4>"))
                    for event in events[:5]:  # Limit to first 5 events
                        display(HTML(f"<p>- {event}</p>"))
            else:
                display(HTML("<p>No plotline data available.</p>"))
        
        # Storyline tab
        with tab.children[2]:
            if analyzer.storyline:
                # Display storyline visualization
                storyline_img = analyzer.visualize_storyline()
                if storyline_img:
                    display(HTML("<h3>Character Network Visualization</h3>"))
                    display(Image.open(storyline_img))
                
                # Display characters
                display(HTML("<h3>Characters</h3>"))
                for character, info in analyzer.storyline["characters"].items():
                    display(HTML(f"<p><b>{character}</b>: mentioned {info['mentions']} times</p>"))
                
                # Display themes
                display(HTML("<h3>Themes</h3>"))
                for theme in analyzer.storyline["themes"]:
                    display(HTML(f"<p>- {theme}</p>"))
                
                # Display settings
                if analyzer.storyline["settings"]:
                    display(HTML("<h3>Settings</h3>"))
                    for setting in analyzer.storyline["settings"]:
                        display(HTML(f"<p>- {setting}</p>"))
                
                # Display conflicts
                if analyzer.storyline["conflicts"]:
                    display(HTML("<h3>Conflicts</h3>"))
                    for conflict in analyzer.storyline["conflicts"]:
                        display(HTML(f"<p>- {conflict}</p>"))
            else:
                display(HTML("<p>No storyline data available.</p>"))
    
    def on_download_button_click(b):
        with status_output:
            clear_output()
            print("Generating report...")
            
            # Generate the report
            report_path = analyzer.generate_report()
            
            if report_path:
                print("Report generated successfully.")
                
                # Create a download link
                display(FileLink(report_path, result_html_prefix="Click here to download the report: "))
            else:
                print("Error: Failed to generate report.")
    
    def on_help_button_click(b):
        with help_output:
            clear_output()
            display(HTML("""
            <h3>How to Use This Tool</h3>
            <p><strong>1. File Format:</strong> Upload a JSON file containing narrative text.</p>
            <p>The JSON file should have one of these structures:</p>
            <pre>
            {
              "text": "Your narrative text here..."
            }
            </pre>
            <p>OR</p>
            <pre>
            {
              "content": "Your narrative text here..."
            }
            </pre>
            <p>OR</p>
            <pre>
            {
              "title": "Story Title",
              "text": "Your narrative text here..."
            }
            </pre>
            
            <p><strong>2. Analysis:</strong> Click "Analyze Text" to process the narrative.</p>
            <p><strong>3. Results:</strong> View the analysis in the tabs below.</p>
            <p><strong>4. Report:</strong> Click "Download Report" to get a PDF report of the analysis.</p>
            
            <h4>Sample JSON</h4>
            <pre>
            {
              "title": "The Three Little Pigs",
              "text": "Once upon a time, there were three little pigs. The first pig built a house of straw. The second pig built a house of sticks. The third pig built a house of bricks. One day, a big bad wolf came to the first pig's house. He huffed and puffed and blew the house down. The first pig ran to the second pig's house. The wolf followed and blew down the second house too. Both pigs ran to the third pig's house. The wolf tried to blow down the brick house, but he couldn't. He tried to enter through the chimney, but the third pig had a pot of boiling water. The wolf fell into the water and ran away. The three pigs lived happily ever after."
            }
            </pre>
            """))
    
    # Register callbacks
    file_upload.observe(on_file_upload_change, names='value')
    analyze_button.on_click(on_analyze_button_click)
    download_button.on_click(on_download_button_click)
    help_button.on_click(on_help_button_click)
    
    # Layout the widgets
    upload_box = widgets.HBox([file_upload, analyze_button, download_button, help_button])
    
    # Display the GUI
    display(header)
    display(upload_box)
    display(status_output)
    display(help_output)
    display(tab)
    display(result_output)

# Run the application
create_narrative_analysis_gui()

 


HTML(value='<h1>Computational Narrative Analysis</h1><p>Upload a JSON file containing narrative text for analy…

Output()

Output()

Output()

This tool can be very useful for literary analysis, storytelling research, or educational purposes. Users can now upload JSON files containing narrative text and get detailed insights into the timeline, plotline, and storyline structures.

If you want to further enhance the tool in the future, you might consider:

Adding support for more file formats (TXT, DOCX, etc.)
Implementing more advanced NLP techniques for better event extraction
Adding sentiment analysis to track emotional arcs in narratives
Creating more sophisticated visualizations
Adding the ability to compare multiple narratives